In [ ]:
# Look-Up-Table-Revised - Scenario 36 - 64 Beams!

In [ ]:
import os
import time
import csv
import copy
import utm
import shutil
import datetime
import itertools
import numpy as np
import pandas as pd
import scipy.io as scipyio

In [ ]:
# Functions

In [ ]:
def get_experiment_name(scen_idx, n_beams, norm_type, noise):
    return f'scenario {scen_idx} beams {n_beams} norm {norm_type} noise {noise}'


def min_max(arr, ax=0):
    """ Computes min-max normalization of array <arr>. """
    return (arr - arr.min(axis=ax)) / (arr.max(axis=ax) - arr.min(axis=ax))


def xy_from_latlong(lat_long):
    """ Assumes lat and long along row. Returns same row vec/matrix on
    cartesian coords."""
    # utm.from_latlon() returns: (EASTING, NORTHING, ZONE_NUMBER, ZONE_LETTER)
    x, y, *_ = utm.from_latlon(lat_long[:,0], lat_long[:,1])
    return np.stack((x,y), axis=1)


def add_pos_noise(pos, noise_variance_in_m=1):

    n_samples = pos.shape[0]

    # Get noise in xy coordinates
    dist = np.random.normal(0, noise_variance_in_m, n_samples)
    ang = np.random.uniform(0, 2*np.pi, n_samples)
    xy_noise = np.stack((dist * np.cos(ang), dist * np.sin(ang)), axis=1)

    # Get position in xy coordinates
    x, y, zn, zl = utm.from_latlon(pos[:,0], pos[:,1])
    xy_pos = np.stack((x,y), axis=1)

    # Apply noise to position and return conversion to lat_long coordinates
    xy_pos_noise = xy_pos + xy_noise

    lat,long = utm.to_latlon(xy_pos_noise[:,0], xy_pos_noise[:,1], zn, zl)
    pos_with_noise = np.stack((lat,long), axis=1)
    return pos_with_noise


def normalize_pos(pos1, pos2, norm_type):
    if norm_type == 1:
        pos_norm = min_max(pos2)

    if norm_type == 2:
        # Check where the BS is and flip axis
        pos_norm = min_max(pos2)

        avg_pos2 = np.mean(pos2, axis=0)

        if pos1[0,0] > avg_pos2[0]:
            pos_norm[:,0] = 1 - pos_norm[:,0]
        if pos1[0,1] > avg_pos2[1]:
            pos_norm[:,1] = 1 - pos_norm[:,1]

    if norm_type == 3:
        pos_norm = min_max(xy_from_latlong(pos2))


    if norm_type  == 4:
        # For relative positions, rotate axis, and min_max it.
        pos2_cart = xy_from_latlong(pos2)
        pos_bs_cart = xy_from_latlong(pos1)
        avg_pos2 = np.mean(pos2_cart, axis=0)

        vect_bs_to_ue = avg_pos2 - pos_bs_cart

        theta = np.arctan2(vect_bs_to_ue[1], vect_bs_to_ue[0])
        rot_matrix = np.array([[ np.cos(theta), np.sin(theta)],
                               [-np.sin(theta), np.cos(theta)]])
        pos_transformed =  np.dot(rot_matrix, pos2.T).T
        pos_norm = min_max(pos_transformed)


    if norm_type == 5:
        pos2_cart = xy_from_latlong(pos2)
        pos_bs_cart = xy_from_latlong(pos1)
        pos_diff = pos2_cart - pos_bs_cart

        # get distances and angle from the transformed position
        dist = np.linalg.norm(pos_diff, axis=1)
        ang = np.arctan2(pos_diff[:,1], pos_diff[:,0])

        # Normalize distance + normalize and offset angle
        dist_norm = dist / max(dist)

        # 1- Get the angle to the average position
        avg_pos = np.mean(pos_diff, axis=0)
        avg_pos_ang = np.arctan2(avg_pos[1], avg_pos[0])

        # A small transformation to the angle to avoid having breaks
        # between -pi and pi
        ang2 = np.zeros(ang.shape)
        for i in range(len(ang)):
            ang2[i] = ang[i] if ang[i] > 0 else ang[i] + 2 * np.pi

        avg_pos_ang2 = \
            avg_pos_ang + 2 * np.pi if avg_pos_ang < 0 else avg_pos_ang

        # 2- Offset angle avg position at 90º
        offset2 = np.pi/2 - avg_pos_ang2
        ang_final = ang2 + offset2

        # MAP VALUES OF 0-PI TO 0-1
        ang_norm = ang_final / np.pi

        pos_norm = np.stack((dist_norm,ang_norm), axis=1)

    return pos_norm


def mode_list(arr):
    """ Returns ordered list based on # of occurences in 1D array. """
    vals, counts = np.unique(arr, return_counts=True)
    return vals[np.flip(np.argsort(counts))]


def pos_to_bin(pos, bin_size, n_bins):
    # The bin indices will be flattened out
    #
    # x2
    # ^
    # | d e f
    # | a b c
    # --------> x1
    #
    # Will be mapped to: a b c d e f
    if pos[0] == 1:
        pos[0] -= 1e-9

    if pos[1] == 1:
        pos[1] -= 1e-9

    bin_idx = int(np.floor(pos[0] / bin_size[0]) +
                  1 / bin_size[0] * np.floor(pos[1] / bin_size[1]))

    return max(min(bin_idx, n_bins-1), 0)


def write_results_together(ai_strategy, top_beams, runs_folder, n_runs,
                           val_accs, test_accs, mean_power_losses):
    results_file = os.path.join(runs_folder, f'{n_runs}-runs_results_summary.txt')
    with open(results_file, 'w') as fp:
        fp.write('Test Results: \n')
        # For test accuracy results
        for i in range(len(top_beams)):
            s = f'Top-{top_beams[i]} average accuracy ' + \
                 f'{np.mean(test_accs[:,i]):.2f} % '
            print(s, end='')
            fp.write(s)
        fp.write('\n')
        # For test Power loss results.
        fp.write('Power Loss results\n')
        fp.write(f"Mean:{np.mean(mean_power_losses):.2f}, "
                 f"STD: {np.std(mean_power_losses):.4f} ")


def write_results_separate(top_beams, results_folder, n_runs,
                           val_accs, test_accs, mean_power_losses):
    variables = [val_accs, test_accs, mean_power_losses]

    for idx, var in enumerate(variables):
        if idx != 2: # (power loss doesn't have top-X results)
            for i, top_beam in enumerate(top_beams):
                mean_and_std = np.array([np.mean(var[:,i]), np.std(var[:,i])])
                acc_str = 'val' if idx == 0 else 'test'
                fname = os.path.join(results_folder,
                                     f'{n_runs}-runs_top-{top_beam}_'
                                     f'{acc_str}_acc.txt',)
                np.savetxt(fname, mean_and_std, fmt='%.2f')
        else:
            mean_and_std = np.array([np.mean(var), np.std(var)])
            fname = os.path.join(results_folder,
                                 f'{n_runs}-runs_mean_power_loss_db.txt')
            np.savetxt(fname, mean_and_std, fmt='%.2f')

In [ ]:
# Settings and hyperparameters

In [ ]:
# Where every result folder will be saved too:
save_folder =  os.path.join(os.getcwd(), f'saved_folder/results_{time.time()}')

# Variables to loop
norm_types = [1]                     # [1,2,3,4,5]
scen_idxs = [1] #np.arange(1,9+1)   # [1,2,3,4,5,6,7,9]
n_beams_list = [64]          # [8, 16,32,64]
noises = [0]                         # position noise in meters
n_reps = 5 #5                           # number repetitions of current settings.

# Variables constant across simulation
max_samples = 1e9                  # max samples to consider per scenario
n_avgs = 5 # 5                            # number of runs to average
train_val_test_split = [60,20,20]     # 0 on val uses test set to validate.
top_beams = np.arange(15) + 1          # Collect stats for Top X predictions
force_seed = -1                       # When >= 0, sets data randimzation
                                      # seed. Useful for data probing.
                                      # Otherwise, seed = run_idx.
# Hyperparameters:
n_lookuptable = 25                    # number of divisions of each coordinate
use_best_n_lookuptable = False         # if True, ignores the value above.
BEST_N_PER_SCENARIO_TAB = \
    [62,40,27,22,30,33,27,20,27]      # best n measured in each scenario

In [ ]:
# Main Part

In [ ]:
combinations = list(itertools.product(scen_idxs, n_beams_list, norm_types,
                                      noises, [1 for i in range(n_reps)]))

for scen_idx, n_beams, norm_type, noise, rep in combinations:

    print(f'Executing for scen={scen_idx}, beams={n_beams}, norm={norm_type}')

    # data_folder = os.path.join(os.getcwd(), f'Ready_data_norm{norm_type}')
    
    data_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))

    # The saved folder will have all experiments conducted.
    experiment_name = get_experiment_name(scen_idx, n_beams, norm_type, noise)
    saved_path = os.path.join(save_folder, experiment_name)


    if not os.path.isdir(save_folder):
        os.makedirs(save_folder)

    if not os.path.isdir(saved_path):
        os.mkdir(saved_path)

    # ----------------------- Phase 1: Data Loading ---------------------------

    ##
    
    csv_file_path = os.path.abspath('scenario36_64_pos_beam.csv')
    data = pd.read_csv(csv_file_path)

    pos1_paths = np.array(data["original_unit1_gps1"])
    pos2_paths = np.array(data["original_unit2_gps1"])
    pwr1_paths = np.array(data["original_unit1_pwr1"])

    # pos1
    pos1 = []
    for path in pos1_paths:
        file_path = os.path.join(data_folder, 'Scenario36', path.split('/', 1)[1])
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                pos1.append([float(line.strip()) for line in file])
        else:
            print(f"File not found: {file_path}")

    pos1 = np.array(pos1)

    # pos2
    pos2 = []
    for path in pos2_paths:
        file_path = os.path.join(data_folder, 'Scenario36', 'unit2', path.split('/', 2)[2])
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                pos2.append([float(line.strip()) for line in file])
        else:
            print(f"File not found: {file_path}")
    pos2 = np.array(pos2)
    
    # pwr1
    pwr1 = []
    for path in pwr1_paths:
        file_path = os.path.join(data_folder, 'Scenario36', 'unit1', path.split('/', 2)[2])
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                pwr1.append([float(line.strip()) for line in file])
        else:
            print(f"File not found: {file_path}")

    pwr1 = np.array(pwr1)

    print(pwr1.shape)
    ##

    max_beams = pwr1.shape[-1]

    n_samples = min(len(pos2), max_samples)
    
   # ========= Phase *: Using csv files for val, train, test data  =============
    data_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))
    
    
    origin_data_path = os.path.join(data_folder, 'Scenario36', 'scenario36.csv')
    
    test_data_path = os.path.abspath('scenario36_64_pos_beam_test.csv')
    train_data_path = os.path.abspath('scenario36_64_pos_beam_train.csv')
    val_data_path = os.path.abspath('scenario36_64_pos_beam_val.csv')
    
    
    origin_data = pd.read_csv(origin_data_path)
    
    test_data = pd.read_csv(test_data_path)
    train_data = pd.read_csv(train_data_path)
    val_data = pd.read_csv(val_data_path)
    
    
    origin_index = np.array(origin_data["abs_index"])

    test_index = np.array(test_data["original_index"])
    train_index = np.array(train_data["original_index"])
    val_index = np.array(val_data["original_index"])
    
    
    #=========- updating indices of index array because original index is not continuous =========
    # convert index array into list
    origin_index_lst = list(origin_index)
    
    # test
    for i in range(len(test_index)):
        test_index[i] = origin_index_lst.index(test_index[i])
    
    # train
    for i in range(len(train_index)):
        train_index[i] = origin_index_lst.index(train_index[i])
    
    # val
    for i in range(len(val_index)):
        val_index[i] = origin_index_lst.index(val_index[i])

    #=============================================================================================
    
    
    test_pwr1_paths = np.array(test_data["original_unit1_pwr1"])
    train_pwr1_paths = np.array(train_data["original_unit1_pwr1"])
    val_pwr1_paths = np.array(val_data["original_unit1_pwr1"])
    
    
    # test
    
    test_pwr1 = []
    for path in test_pwr1_paths:
        file_path = os.path.join(data_folder, 'Scenario36', 'unit1', path.split('/', 2)[2])
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                test_pwr1.append([float(line.strip()) for line in file])
        else:
            print(f"File not found: {file_path}")

    test_pwr1 = np.array(test_pwr1)
    top_test_data = np.argmax(test_pwr1, axis=1)
    
    # train
    
    train_pwr1 = []
    for path in train_pwr1_paths:
        file_path = os.path.join(data_folder, 'Scenario36', 'unit1', path.split('/', 2)[2])
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                train_pwr1.append([float(line.strip()) for line in file])
        else:
            print(f"File not found: {file_path}")

    train_pwr1 = np.array(train_pwr1)
    top_train_data = np.argmax(train_pwr1, axis=1)
    
    # val
    
    val_pwr1 = []
    for path in val_pwr1_paths:
        file_path = os.path.join(data_folder, 'Scenario36', 'unit1', path.split('/', 2)[2])
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                val_pwr1.append([float(line.strip()) for line in file])
        else:
            print(f"File not found: {file_path}")

    val_pwr1 = np.array(val_pwr1)
    top_val_data = np.argmax(val_pwr1, axis=1)

    # -------------------- Phase 2: Data Preprocessing ------------------------

    # Trim altitudes if they exists
    pos1 = pos1[:,:2]
    pos2 = pos2[:,:2]

    # Insert Noise if enabled.
    pos2_with_noise = add_pos_noise(pos2, noise_variance_in_m=noise)

    # Normalize position
    pos_norm = normalize_pos(pos1, pos2_with_noise, norm_type)

    # Assign beam values (and automatically downsample if n_beams != 64)
    if n_beams not in [8, 16, 32, 64]:
        raise Exception('')

    divider = max_beams // n_beams
    beam_idxs = np.arange(0, max_beams, divider)

    # Select alternating samples (every 2, 4 or 8)
    beam_pwrs = pwr1[:,beam_idxs]
    # Convert beam indices. 32 beams: ([0,2,4,..., 62]) -> [0,1,2,... 32]
    beam_data = np.argmax(beam_pwrs, axis=1)

    # ----------------- Phase 3: Define Path for run --------------------------

    # We first define the folder where results from this run will be saved
    # In that folder there will be other runs too, and that will tell us what's
    # the index of this run. That information is used to shuffle the data
    # in a reproducible way. Run 1 uses seed 1, run 2 uses seed 2, etc.

    # Compute parameters needed for setting runs_folder name
    # The Runs Folder contains the folders of each run.

    if use_best_n_lookuptable:
        n = BEST_N_PER_SCENARIO_TAB[scen_idx-1]
    else:
        n = n_lookuptable

    runs_folder_name = f'LT_N={n}'
    runs_folder = os.path.join(saved_path, runs_folder_name)

    # Create if doesn't exist
    if not os.path.isdir(runs_folder):
        os.mkdir(runs_folder)

    # Experiment index: number of experiments already conducted + 1
    run_idx = 1 + sum(os.path.isdir(os.path.join(runs_folder, run_folder))
                        for run_folder in os.listdir(runs_folder))

    now_time = datetime.datetime.now().strftime('Time_%m-%d-%Y_%Hh-%Mm-%Ss')

    run_folder = os.path.join(runs_folder, f"{run_idx}-{now_time}")

    # Check if there are enough runs. If yes, skip data loading, model
    # training and testing, and jump to averaging the results.
    
    if run_idx > n_avgs:
        print('Already enough experiments conducted for '
                'this case. Either increase n_avgs, or try '
                'a different set of parameters. SKIPPING TO the avg. '
                'computation!')
    else:
        # -------------------- Phase 4: Split Data ------------------------

        # Create folder for the run
        os.mkdir(run_folder)

        # Shuffle Data (fixed, for reproducibility)
        if force_seed >= 0:
            np.random.seed(force_seed)
        else:
            np.random.seed(run_idx)
        sample_shuffle_list = np.random.permutation(n_samples)

        # Select sample indices for each set
        first_test_sample = int((1-train_val_test_split[2]) / 100 * n_samples)
        train_val_samples = sample_shuffle_list[:first_test_sample]
        test_samples = sample_shuffle_list[first_test_sample:]

        # (We have no use for x_val in KNN or LookupTable.)
        # CHOICE: we used train+val(80%) sets to train in KNN and the LUtable
        #         but in the NN we used only train(60%). This seemed a fair
        #         approach, and doing otherwise yields little difference.
        if train_val_test_split[1] == 0:
            train_samples = train_val_samples
            val_samples = test_samples
        else:
            train_ratio = np.sum(train_val_test_split[:2]) / 100
            first_val_sample = int(train_val_test_split[1] / 100 *
                                    len(train_val_samples) / train_ratio)
            val_samples = train_val_samples[:first_val_sample]
            train_samples = train_val_samples[first_val_sample:]

        x_train = pos_norm[train_samples]
        y_train = beam_data[train_samples]
        x_val = pos_norm[val_samples]
        y_val = beam_data[val_samples]
        x_test = pos_norm[test_samples]
        y_test = beam_data[test_samples]
        y_test_pwr = beam_pwrs[test_samples]
         
        
        #=========== updated code by csv file ================
        
        new_x_train = pos_norm[train_index]
        new_y_train = top_train_data
        new_x_val = pos_norm[val_index]
        new_y_val = top_val_data
        new_x_test = pos_norm[test_index]
        new_y_test = top_test_data
        new_y_test_pwr = beam_pwrs[test_samples]
        
        #======================================================

    # ---------------------- Phase 5: Train & Test ------------------------

        # Useful little variables
        n_test_samples = len(new_x_test)
        n_top_stats = len(top_beams)

        # Variables for compatibility (when not all predictors are used)
        n_bins, bin_size, prediction_per_bin = None, None, None
        trained_model = None


    if run_idx <= n_avgs:

        # 1- Define bins

        n_bins_across_x1 = 8
        n_bins_across_x2 = 8
        bin_size = np.array([1,1]) / [n_bins_across_x1, n_bins_across_x2]
        n_bins = n_bins_across_x1 * n_bins_across_x2
#         n_bins = 65

        # 2- Create a list with the samples per bin
        samples_per_bin = [[] for bin_idx in range(n_bins)]

        # 3- Map each input to a bin
        for x_idx, x in enumerate(new_x_train):
            samples_per_bin[pos_to_bin(x, bin_size, n_bins)].append(x_idx)

        # 4- Define the values to predict for samples in that bin
        prediction_per_bin = [mode_list(new_y_train[samples_per_bin[bin_idx]])
                                for bin_idx in range(n_bins)]

        
        # 5- Evaluation phase. Map each test sample to a bin and get the
        #    prediction.
        pred_beams = []
        for x in x_test:
            pred = prediction_per_bin[pos_to_bin(x, bin_size, n_bins)]
            if pred.size == 0:
                pred = [int(np.random.uniform(0, n_beams))]
            pred_beams.append(np.asarray(pred))
            
        pred_values = []
        for x in new_x_test:
            pred = prediction_per_bin[pos_to_bin(x, bin_size, n_bins)]
            if pred.size == 0:
                pred = [int(np.random.uniform(0, n_beams))]
            pred_values.append(np.asarray(pred))
        print(len(pred_values[0]))

    #=========== Add Top-1 ~ Top-15 columns into test csv file ===========
        # making only Top-1, Top-3, ... array
        top_1_data = [each_data[0] for each_data in pred_values]
        top_3_data = [each_data if len(each_data) < 3 else each_data[:3] for each_data in pred_values]
        top_5_data = [each_data if len(each_data) < 5 else each_data[:5] for each_data in pred_values]
        top_7_data = [each_data if len(each_data) < 7 else each_data[:7] for each_data in pred_values]
        top_9_data = [each_data if len(each_data) < 9 else each_data[:9] for each_data in pred_values]
        top_11_data = [each_data if len(each_data) < 11 else each_data[:11] for each_data in pred_values]
        top_13_data = [each_data if len(each_data) < 13 else each_data[:13] for each_data in pred_values]
        top_15_data = [each_data if len(each_data) < 15 else each_data[:15] for each_data in pred_values]
        
        
        # Add Top-1, Top-3,... columns
        test_data["Top-1"] = ""
        test_data["Top-1"] = top_1_data
        
        test_data["Top-3"] = ""
        test_data["Top-3"] = top_3_data
        
        test_data["Top-5"] = ""
        test_data["Top-5"] = top_5_data
        
        test_data["Top-7"] = ""
        test_data["Top-7"] = top_7_data
        
        test_data["Top-9"] = ""
        test_data["Top-9"] = top_9_data
        
        test_data["Top-11"] = ""
        test_data["Top-11"] = top_11_data
        
        test_data["Top-13"] = ""
        test_data["Top-13"] = top_13_data
        
        test_data["Top-15"] = ""
        test_data["Top-15"] = top_15_data
        
        # update test csv file
        test_data.to_csv(test_data_path, index=False)
    
    #=====================================================================
    
    
    # ----------- Phase 6: Save Accuracies and Power Losses ---------------

        # Get top-1, top-2, top-3 and top-5 accuracies
        total_hits = np.zeros(n_top_stats)

        # For each test sample, count times where true beam is in top 1,2,3,5
        for i in range(n_test_samples):
            for j in range(n_top_stats):
                hit = np.any(pred_beams[i][:top_beams[j]] == y_test[i])
                total_hits[j] += 1 if hit else 0

        # Average the number of correct guesses (over the total samples)
        acc = np.round(total_hits / n_test_samples, 4)

        print(f'LT Results:')
        for i in range(n_top_stats):
            print(f'\tAverage Top-{top_beams[i]} accuracy {acc[i]*100:.2f}')

        # Save Test acc to file
        np.savetxt(os.path.join(run_folder, 'test_accs.txt'),
                    acc * 100, fmt='%.2f')

        # We consider the noise per sample, not per scenario:
        # Noise is the lowest power in each sample.
        power_loss_ratio = np.zeros(n_test_samples)
        for i in range(n_test_samples):
            noise = np.min(y_test_pwr[i,:]) / 2

            pred_pwr = y_test_pwr[i,pred_beams[i][0]]
            true_pwr = np.max(y_test_pwr[i,:])

            if pred_pwr == noise:
                # In Lookup table it may be the case we predict the worst
                # beam to be the best. In this case, adjust noise slightly
                # just to avoid -inf dB loss. This is extremely rare and
                # will not affect results noticeably.
                noise = noise/2

            power_loss_ratio[i] = ((true_pwr - noise) /
                                    (pred_pwr - noise))

        mean_power_loss_db = 10 * np.log10(np.mean(power_loss_ratio))

        print(f"{mean_power_loss_db:.4f}")

        np.savetxt(os.path.join(run_folder, 'power_loss.txt'),
                    np.stack((mean_power_loss_db, 0))) # needs to be 1D..



        # -------------- Phase 7: Compute average across runs ----------------
    if run_idx >= n_avgs:
        folders_of_each_run = [os.path.join(runs_folder, folder)
                                for folder in os.listdir(runs_folder)]

        folders_of_each_run = [folder for folder in folders_of_each_run
                                if os.path.isdir(folder)]

        n_run_folders = len(folders_of_each_run)
        val_accs = np.zeros((n_run_folders, len(top_beams)))
        test_accs = np.zeros((n_run_folders, len(top_beams)))
        mean_power_losses = np.zeros(n_run_folders)
        for run_idx in range(n_run_folders):
            run_folder = folders_of_each_run[run_idx]
            test_accs_file = os.path.join(run_folder, 'test_accs.txt')
            pwr_loss_file = os.path.join(run_folder, 'power_loss.txt')

            test_accs[run_idx] = np.loadtxt(test_accs_file)
            mean_power_losses[run_idx] = np.loadtxt(pwr_loss_file)[0]

        print(f'Computing the average of {n_run_folders} runs. ')

        # Write results to same text file
        write_results_together('LT', top_beams, runs_folder,
                                    n_run_folders, val_accs,
                                    test_accs, mean_power_losses)

        write_results_separate(top_beams, runs_folder, n_run_folders,
                                    val_accs, test_accs, mean_power_losses)